In [ ]:
import os
import math
import torch
from torch.autograd import Variable
from torch.optim import Adam
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image

In [ ]:
BATCH_SIZE = 20
LABELS = sorted([i[:-4] for i in os.listdir('icons')])
NUM_CLASSES = len(LABELS)

In [ ]:
class IconsDataset(Dataset):
    def __init__(self, directory, labels, transform=None):
        self.directory = directory
        self.files = os.listdir(directory)
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        f = self.files[index]
        # image
        image = Image.open(os.path.join(self.directory, f))
        if self.transform:
            image = self.transform(image)
        
        # labels
        label_strings = f[:-4].split('_')
        labels = []
        for l in LABELS:
            if l in label_strings:
                labels.append(1)
            else:
                labels.append(0)
        
        return image, torch.tensor(labels, dtype=torch.float)
    
    def input_size(self):
        return 100 * 100 * 3
    
    def __len__(self):
        return len(self.files)


def load_dataset():
    d = IconsDataset('images/', LABELS, transform=transforms.ToTensor())
    train, validate = random_split(d, [800, 200])
    
    loader = DataLoader(train, batch_size=BATCH_SIZE)
    validation_loader = DataLoader(validate, batch_size=BATCH_SIZE)

    return d.input_size(), loader, validation_loader

In [ ]:
class ModelConv1(nn.Module):
    def __init__(self, kernel_size=5, conv_out_channels=5, linear_size=50):
        super().__init__()
        if kernel_size % 2 != 1:
            raise Exception('Only odd kernel_size are supported')
        self.conv_out_channels = conv_out_channels
        self.conv1 = nn.Conv2d(3, conv_out_channels, kernel_size=kernel_size)
        # convolution kernels are not applied on the border of the image, because the kernel would be outside the image
        conv_layer_output_size = int(100 - (kernel_size - 1))
        self.pooled_pixels = int(conv_layer_output_size / 2)
        self.h1 = nn.Linear(self.pooled_pixels * self.pooled_pixels  * conv_out_channels, linear_size)
        self.h2 = nn.Linear(linear_size, NUM_CLASSES)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, self.pooled_pixels * self.pooled_pixels * self.conv_out_channels)
        x = self.h1(x)
        x = F.relu(x)
        x = self.h2(x)
        x = F.softmax(x, dim=1)
        return x

In [ ]:
from datetime import datetime

def evalulate(model, validation_loader):
    model.eval()
    criterion = nn.MultiLabelSoftMarginLoss()
    
    total_loss = 0.0
    for data, labels in validation_loader:
        predictions = model(data.cuda())
        loss = criterion(predictions, labels.cuda())    
        total_loss += loss.data.mean()
    return total_loss/len(validation_loader)

def learn(model, loader, validation_loader, epochs=30, learning_rate=0.001):
    torch.cuda.empty_cache()
    optimizer = Adam(params=model.parameters(), lr=learning_rate)
    criterion = nn.MultiLabelSoftMarginLoss()

    f = open(f'{datetime.now().isoformat()}.txt', 'w', buffering=1)

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for data, labels in loader:
            predictions = model(data.cuda())
            loss = criterion(predictions, labels.cuda())
            
            total_loss += loss.data.mean()

            model.zero_grad()
            loss.backward()
            optimizer.step()
        
        train_loss = total_loss/len(loader)
        validation_loss = evalulate(model, validation_loader)
        stats = f'Epoch: {epoch}, Train Loss: {train_loss}, Validation Loss: {validation_loss.item()}'
        print(stats)
        f.write(f'{stats}\n')
        
    return model

In [ ]:
input_size, loader, validation_loader = load_dataset()
model = ModelConv1(kernel_size=5, conv_out_channels=5, linear_size=50).cuda()
learn(model, loader, validation_loader, epochs=10000, learning_rate=0.001)
